In [1]:
import math
import scipy as sp
import matplotlib.pyplot as plt
from scipy.stats import bernoulli, uniform, chi2
import numpy as np
from scipy.stats.mstats import gmean
from numpy.testing import assert_allclose
from utils import sprt_mart, alpha_mart, shrink_trunc, stratum_selector, multinomial_selector, get_global_pvalue
np.random.seed(123456789)

In [2]:
theta = 1/2 
hand_tally = np.concatenate((np.ones(40), np.zeros(40), np.ones(60), np.zeros(40)))
reported_tally = np.concatenate((np.ones(40), np.zeros(40), np.ones(70), np.zeros(30)))
omega = reported_tally - hand_tally
strata = np.concatenate((np.ones(80), 2*np.ones(100)))


v = np.array([2 * np.mean(reported_tally[strata == 1]) - 1, 2 * np.mean(reported_tally[strata == 2]) - 1])
u = 2 / (2 - v)
stratum_1 = (1 - omega[strata == 1]) / (2 - v[0])
stratum_2 = (1 - omega[strata == 2]) / (2 - v[1])

In [3]:
p_value, stratum_selections, null_selections = get_global_pvalue(
    strata = [stratum_1, stratum_2], 
    u = u, 
    v = v, 
    rule = multinomial_selector)

IndexError: index 179 is out of bounds for axis 0 with size 179

In [5]:
p_value.shape

(179, 180)

In [15]:
strata = [stratum_1, stratum_2]
rule = multinomial_selector

shuffled_1 = np.random.permutation(strata[0])
shuffled_2 = np.random.permutation(strata[1])
N = np.concatenate((np.array([len(shuffled_1)]), np.array([len(shuffled_2)])))
w = N/sum(N)
epsilon = 1 / (2*np.max(N))
theta_1_grid = np.arange(epsilon, u[0] - epsilon, epsilon) #sequence from epsilon to u[0] - epsilon
theta_2_grid = (1/2 - w[0] * theta_1_grid) / w[1]
strata_matrix = np.zeros((len(shuffled_1) + len(shuffled_2) - 1, len(theta_1_grid)))
intersection_marts = np.zeros((len(shuffled_1) + len(shuffled_2), len(theta_1_grid)))
for i in range(len(theta_1_grid)):
    mart_1, mu_1 = alpha_mart(x = shuffled_1, N = N[0], mu = theta_1_grid[i], eta = 1/(2-v[0]), f = .01, u = u[0])
    mart_2, mu_2 = alpha_mart(x = shuffled_2, N = N[1], mu = theta_2_grid[i], eta = 1/(2-v[1]), f = .01, u = u[1])
    strata_matrix[:,i], intersection_marts[:,i] = stratum_selector(
        marts = [mart_1, mart_2],
        mu = [mu_1, mu_2],
        u = u,
        rule = rule)
null_index = np.argmin(intersection_marts, axis = 1)
minimized_martingale = intersection_marts[range(N),null_index]
p_value = 1 / np.maximum(1, minimized_martingale)
stratum_selections = strata_matrix[1:N,null_index]
null_selections = theta_1_grid[null_index]

TypeError: only integer scalar arrays can be converted to a scalar index